In [1]:
!pip install groq
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 11.3 MB/s eta 0:00:00


In [ ]:
import os
import re
import pandas as pd
from groq import Groq

# Set up your Groq API key
os.environ["GROQ_API_KEY"] = ""  # Replace with your actual API key

# Initialize the Groq client
client = Groq()

# Function to generate synthetic text using multiple examples as context
def generate_synthetic_text(input_examples):
    # Format the examples into a prompt context
    context = "\n".join([f"- {text}" for text in input_examples])
    prompt = f"Based on the following examples of text with a similar sentiment:\n{context}\nGenerate a new example with the same sentiment. Please enclose the example in double quotes."

    # Requesting completion from the Mixtral model
    completion = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    generated_text = ""
    for chunk in completion:
        generated_text += chunk.choices[0].delta.content or ""

    # Extract only the text enclosed in double quotes
    quoted_text = re.search(r'"(.*?)"', generated_text)
    return quoted_text.group(1) if quoted_text else None

# Load your dataset
data_path = '/content/preprocessed_twitter_data_small (1).csv'
data = pd.read_csv(data_path)

# Define the column names
sentiment_column = 'sentiment'  # Replace with the actual sentiment column name
text_column = 'text'  # Replace with the actual text column name

# Create synthetic data for each sentiment class
synthetic_data = []
sentiment_classes = data[sentiment_column].unique()  # Extract unique sentiment classes

for sentiment in sentiment_classes:
    # Select all rows of this sentiment to use as context
    sentiment_data = data[data[sentiment_column] == sentiment][text_column].tolist()

    # Generate 10 synthetic rows for this sentiment class
    for i in range(10):
        synthetic_text = generate_synthetic_text(sentiment_data[:10])  # Use the first 10 examples as context
        if synthetic_text:  # Only add if extraction was successful
            synthetic_data.append({
                "original_text": sentiment_data[i % 10],  # Store each example from the first 10 as original text
                "synthetic_text": synthetic_text,
                "sentiment": sentiment
            })

# Convert synthetic data to DataFrame
synthetic_df = pd.DataFrame(synthetic_data)

# Save synthetic data to CSV in the specified order: original_text, synthetic_text, sentiment
synthetic_df.to_csv('/content/synthetic_data_mixtral.csv', index=False)
print("Synthetic data saved successfully.")


Synthetic data saved successfully.


#  Perplexity, Lexical Diversity and Embedding SImilarity

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the synthetic data (make sure this file path is correct)
synthetic_data_path = '/content/synthetic_data_mixtral.csv'
synthetic_data = pd.read_csv(synthetic_data_path)

# Initialize GPT-2 model for perplexity calculation
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Initialize Sentence Transformer model for embedding similarity
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to calculate perplexity
def calculate_perplexity(text):
    inputs = gpt2_tokenizer(text, return_tensors="pt")
    outputs = gpt2_model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

# Function to calculate lexical diversity, handling empty text
def lexical_diversity(text):
    words = text.split()
    return len(set(words)) / len(words) if len(words) > 0 else 0

# Calculate metrics for each row in the synthetic dataset
perplexities_original = []
perplexities_synthetic = []
lexical_diversities_original = []
lexical_diversities_synthetic = []
embedding_similarities = []

for index, row in synthetic_data.iterrows():
    original_text = row['original_text']
    synthetic_text = row['synthetic_text']

    # Perplexity
    perplexities_original.append(calculate_perplexity(original_text))
    perplexities_synthetic.append(calculate_perplexity(synthetic_text))

    # Lexical Diversity
    lexical_diversities_original.append(lexical_diversity(original_text))
    lexical_diversities_synthetic.append(lexical_diversity(synthetic_text))

    # Embedding Similarity
    original_embedding = embedding_model.encode([original_text])[0]
    synthetic_embedding = embedding_model.encode([synthetic_text])[0]
    similarity = cosine_similarity([original_embedding], [synthetic_embedding])[0][0]
    embedding_similarities.append(similarity)

# Add calculated metrics to the dataframe
synthetic_data['perplexity_original'] = perplexities_original
synthetic_data['perplexity_synthetic'] = perplexities_synthetic
synthetic_data['lexical_diversity_original'] = lexical_diversities_original
synthetic_data['lexical_diversity_synthetic'] = lexical_diversities_synthetic
synthetic_data['embedding_similarity'] = embedding_similarities

# Calculate average metrics grouped by sentiment
grouped_metrics = synthetic_data.groupby('sentiment').agg({
    'perplexity_original': 'mean',
    'perplexity_synthetic': 'mean',
    'lexical_diversity_original': 'mean',
    'lexical_diversity_synthetic': 'mean',
    'embedding_similarity': 'mean'
}).reset_index()

# Display the grouped metrics
print("Comparison of Original and Synthetic Text Metrics by Sentiment Type:")
print(grouped_metrics)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Comparison of Original and Synthetic Text Metrics by Sentiment Type:
    sentiment  perplexity_original  perplexity_synthetic  \
0  Irrelevant          3805.486047             48.908300   
1    Negative           320.174151             99.436890   
2     Neutral           501.630981             49.691963   
3    Positive           247.962406             19.332150   

   lexical_diversity_original  lexical_diversity_synthetic  \
0                    0.949769                     0.962557   
1                    0.948430                     0.974888   
2                    0.964069                     0.897646   
3                    0.983687                     1.000000   

   embedding_similarity  
0              0.146035  
1              0.202392  
2              0.142661  
3              0.175746  


# VADAR Sentiment Analysis

In [5]:
# !pip install vaderSentiment
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Function to calculate sentiment intensity
def sentiment_strength(text):
    return analyzer.polarity_scores(text)['compound']

# Load the synthetic data (make sure this file path is correct)
synthetic_data_path = '/content/synthetic_data_mixtral.csv'
synthetic_data = pd.read_csv(synthetic_data_path)

real_data=pd.read_csv('/content/preprocessed_twitter_data_small (1).csv')

# Apply sentiment strength analysis on synthetic and original data
synthetic_data['sentiment_strength'] = synthetic_data['synthetic_text'].apply(sentiment_strength)
real_data['sentiment_strength'] = real_data['text'].apply(sentiment_strength)

# Compare the average sentiment strength by sentiment type
strength_comparison = synthetic_data.groupby('sentiment')['sentiment_strength'].mean()
print("Average Sentiment Strength Comparison by Sentiment Type:")
print(strength_comparison)


Average Sentiment Strength Comparison by Sentiment Type:
sentiment
Irrelevant   -0.044810
Negative     -0.368089
Neutral      -0.096330
Positive      0.675080
Name: sentiment_strength, dtype: float64


# N Gram Analysis

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Calculate bigram overlap
vectorizer = CountVectorizer(ngram_range=(2, 2))

# Transform the real data to extract bigrams
original_bigrams = vectorizer.fit_transform(real_data['text']).toarray()
synthetic_bigrams = vectorizer.transform(synthetic_data['synthetic_text']).toarray()

# Sum along the rows to get the total count of each bigram across the dataset
original_bigrams_sum = original_bigrams.sum(axis=0)
synthetic_bigrams_sum = synthetic_bigrams.sum(axis=0)

# Calculate overlap as a percentage by taking the minimum count of each bigram and dividing by the maximum
overlap = np.sum(np.minimum(original_bigrams_sum, synthetic_bigrams_sum)) / np.sum(np.maximum(original_bigrams_sum, synthetic_bigrams_sum)) * 100

print("Bigram Overlap Percentage between Original and Synthetic Texts:", overlap)


Bigram Overlap Percentage between Original and Synthetic Texts: 2.273389682308365


# Topic Consistency

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

# Load combined data with original and synthetic texts
file_path = '/content/synthetic_data_mixtral.csv'  # Update with actual path
data = pd.read_csv(file_path)

# Initialize LDA and TF-IDF Vectorizer
n_topics = 5  # Adjust the number of topics as necessary
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
vectorizer = TfidfVectorizer(max_features=1000)

# Function to calculate topic distribution for a specific sentiment type and dataset
def calculate_topic_distribution(data_subset, text_column):
    tfidf_matrix = vectorizer.fit_transform(data_subset[text_column])
    topic_distribution = lda.fit_transform(tfidf_matrix)
    return topic_distribution.mean(axis=0)

# Dictionary to store results
comparison_results = {}

# List of unique sentiments
sentiments = data['sentiment'].unique()

# Analyze each sentiment type
for sentiment in sentiments:
    # Filter data for the current sentiment
    original_data_sentiment = data[data['sentiment'] == sentiment]['original_text']
    synthetic_data_sentiment = data[data['sentiment'] == sentiment]['synthetic_text']

    # Calculate topic distribution for original and synthetic data for this sentiment
    original_topic_dist = calculate_topic_distribution(data[data['sentiment'] == sentiment], 'original_text')
    synthetic_topic_dist = calculate_topic_distribution(data[data['sentiment'] == sentiment], 'synthetic_text')

    # Store results
    comparison_results[sentiment] = {
        'Original Topic Distribution': original_topic_dist,
        'Synthetic Topic Distribution': synthetic_topic_dist
    }

# Display results
for sentiment, dist in comparison_results.items():
    print(f"\nSentiment: {sentiment}")
    print("Original Topic Distribution:", dist['Original Topic Distribution'])
    print("Synthetic Topic Distribution:", dist['Synthetic Topic Distribution'])



Sentiment: Irrelevant
Original Topic Distribution: [0.3565141  0.1786207  0.20403984 0.21300903 0.04781633]
Synthetic Topic Distribution: [0.19644312 0.28371778 0.27860742 0.20425976 0.03697192]

Sentiment: Negative
Original Topic Distribution: [0.41189996 0.1363687  0.12936331 0.19290751 0.12946052]
Synthetic Topic Distribution: [0.12648381 0.22099299 0.30507317 0.30841867 0.03903136]

Sentiment: Neutral
Original Topic Distribution: [0.18886666 0.12060077 0.12064597 0.2027136  0.36717301]
Synthetic Topic Distribution: [0.28201121 0.1187045  0.0369311  0.11781248 0.44454071]

Sentiment: Positive
Original Topic Distribution: [0.13622398 0.22522698 0.29807291 0.12899818 0.21147795]
Synthetic Topic Distribution: [0.196517   0.27603939 0.04660732 0.20406283 0.27677345]


# Diversity of Generated Text

In [8]:
import pandas as pd

# Specify the path to your file containing both original and synthetic data
file_path = '//content/synthetic_data_mixtral.csv'  # Update with actual file path

# Load the data
combined_data = pd.read_csv(file_path)

# Calculate uniqueness within the synthetic data itself
unique_synthetic_texts = combined_data['synthetic_text'].nunique()
total_synthetic_texts = len(combined_data['synthetic_text'])
self_uniqueness_ratio = unique_synthetic_texts / total_synthetic_texts * 100

# Calculate overlap between original and synthetic data
overlapping_texts = combined_data[combined_data['synthetic_text'].isin(combined_data['original_text'])].shape[0]
original_to_synthetic_overlap_ratio = (overlapping_texts / total_synthetic_texts) * 100

# Print the results
print("Uniqueness Ratio of Synthetic Data (compared to itself):", self_uniqueness_ratio, "%")
print("Overlap Ratio between Original and Synthetic Data:", original_to_synthetic_overlap_ratio, "%")


Uniqueness Ratio of Synthetic Data (compared to itself): 100.0 %
Overlap Ratio between Original and Synthetic Data: 0.0 %


# Readability Scores

In [9]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.4 MB/s eta 0:00:00


In [10]:
# Install textstat library

import pandas as pd
from textstat import flesch_reading_ease, flesch_kincaid_grade

# Load the combined dataset containing both original and synthetic text columns
file_path = '/content/synthetic_data_mixtral.csv'  # Update with the actual file path
combined_data = pd.read_csv(file_path)

# Calculate readability for synthetic and original texts separately
combined_data['flesch_reading_ease_original'] = combined_data['original_text'].apply(flesch_reading_ease)
combined_data['flesch_reading_ease_synthetic'] = combined_data['synthetic_text'].apply(flesch_reading_ease)

# Compare average readability by sentiment type for both original and synthetic texts
readability_comparison = combined_data.groupby('sentiment')[
    ['flesch_reading_ease_original', 'flesch_reading_ease_synthetic']
].mean()

# Print the readability comparison by sentiment
print("Average Readability Comparison by Sentiment Type:")
print(readability_comparison)


Average Readability Comparison by Sentiment Type:
            flesch_reading_ease_original  flesch_reading_ease_synthetic
sentiment                                                              
Irrelevant                        70.560                      68.823000
Negative                          63.340                      54.785556
Neutral                           67.858                      69.145000
Positive                          95.657                      72.355000
